## Bitcoin Allocation Strategies
Comparing different allocation strategies historically

. Lump-Sum

. Dollar Cost Averaging (DCA)

. Combination (optimized)

In [54]:
# Import Libraries
import pandas as pd
import numpy as np
import urllib
import requests
import riskfolio as rp
from datetime import datetime, timedelta
from copy import deepcopy

from highcharts import Highchart
from chart_builder import plot_chart, CHART_DEFAULTS
from IPython.display import display, HTML, display_html
import import_ipynb

# Chart libraries + settings
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = None  # default='warn' - disable some pandas warnings
pd.options.display.float_format = '{:,.2f}'.format


In [55]:
# Load Bitcoin Prices into a dataframe
# Ticker is customizable
ticker = "BTC"
# Cryptocompare URL and fiels
base_url = 'https://min-api.cryptocompare.com/data/histoday'
ticker_field = 'fsym'
field_dict = {'tsym': 'USD','allData': 'true'}
# Convert the field dict into a url encoded string
url_args = "&" + urllib.parse.urlencode(field_dict)
ticker = ticker.upper()
globalURL = (base_url + "?" + ticker_field + "=" + ticker + url_args)

In [56]:
# Request the data
resp = requests.get(url=globalURL)
data = resp.json()
data["Response"]

'Success'

In [57]:
# Parse the JSON into a Pandas DataFrame
try:
    df = pd.DataFrame.from_dict(data['Data'])
    df = df.rename(columns={'time': 'date'})
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df_save = df[['close', 'open', 'high', 'low']]
except Exception as e:
    print(e)
    df_save = None

In [58]:
# Include percentage change 
df = df_save
df['change'] = df['close'].pct_change()

### Support Utilities to be used later

In [59]:
# Increment n number of months of certain date
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return new_date

def add_periods(date, periods, frequency):
    if frequency.upper() == 'D' or 'DAY' in frequency.upper():
        return (date + timedelta(days=periods))
    if frequency.upper() == 'W' or 'WEEK' in frequency.upper():
        return (date + timedelta(days=periods * 7))
    if frequency.upper() == 'M' or 'MONTH' in frequency.upper():
        return(monthdelta(date, periods))
    if frequency.upper() == 'Y' or 'YEAR' in frequency.upper():
        return(monthdelta(date, periods * 12))
    
def annualization_factor(df):
    """
    Receives a df and returns the number of periods to apply
    to annualize the returns. For BTC this should be close to
    365 as it trades daily. For stocks should be close to 252.
    Args:
        df (_type_): _description_
    """
    start_date = df.index[0]
    end_date = df.index[-1]
    number_of_days = (end_date - start_date).days
    fraction_of_year = number_of_days / 365
    data_points = len(df)
    annualization_factor = data_points / fraction_of_year
    return int(round(annualization_factor, 0))


### Main Allocation Class
See example on creating an allocation instance at the cell following the class definition

In [60]:
class AllocationManager:
    def __init__(self):
        self.frequency = 'D'  # 'D', 'W', 'M', 'Y'
        self.allocation_periods = 30  # Assume allocation happens during 30 periods
        self.upfront_percent = 0  # [0 - 1]: amount to be allocated upfront
        self.capital = 100000  # 10,000 dollars to allocate
        self.df = df  # Bitcoin Prices Dataframe
        self.start_date = self.df.index.min()  # Date where allocation starts (default = first date)
                                               # ex: pd.to_datetime('2023-01-01', format='%Y-%m-%d')
        self.end_date = self.df.index.max()  # End date for analysis (default = today), but this can be used to test specific timeframes (ex: ending last year)
                                             # ex: pd.to_datetime('2023-01-01', format='%Y-%m-%d')
        # Create empty allocation & position columns
        self.df['allocation'] = 0
        self.df['BTC_tx'] = 0
        self.risk_free_rate = 0.05
         
    def allocate_capital(self):
        # TRIM THE DF between start and end dates
        # Filter the dataframe to only include selected dates
        self.df = self.df[(self.df.index >= (self.start_date)) & (self.df.index <= self.end_date)]
        
        # Clear allocation and other columns
        self.df['allocation'] = 0
        self.df['BTC_tx'] = 0
        self.df['BTC_position'] = 0
        self.df['portfolio_position'] = 0
        self.df['cum_capital'] = 0
        self.df['normalized_port_position'] = 0
        self.df['cum_return'] = 0
        
        # Updates the dataframe to allocate the capital
        current_date = self.start_date
        periods_left = self.allocation_periods
        
        # Set upfront amount if any & per period amounts
        if self.upfront_percent > 0:
            upfront = self.upfront_percent * self.capital  # how much upfront in $
            per_period = (self.capital - upfront) / (self.allocation_periods - 1)
        else:
            per_period = self.capital / self.allocation_periods 
            upfront = per_period
        
        # Start looping until allocation is complete
        while periods_left > 0:
            # Allocate Capital
            if current_date == self.start_date:
                self.df.at[current_date, 'allocation'] = upfront
            else:
                self.df.at[current_date, 'allocation'] = per_period
            
            # Allocate BTC
            self.df.at[current_date, 'BTC_tx'] = (
                self.df.at[current_date, 'allocation'] / 
                self.df.at[current_date, 'close'] 
                )
            current_date = add_periods(current_date, 1, self.frequency)
            if current_date > self.end_date:
                raise Exception("Allocation dates overflow end date. Either shorten the allocation period or increase the data range.")
            periods_left -= 1

        # Sum all BTC Txs and calculate portfolio values
        self.df['BTC_position'] = self.df['BTC_tx'].cumsum()
        self.df['portfolio_position'] = (self.df['BTC_position'] * self.df['close'])
        self.df = self.df.fillna(0)
        # TO DO ---------------------
        self.df['cum_capital'] = self.df['allocation'].cumsum()
        self.df['normalized_port_position'] = (self.df['portfolio_position'] / 
                                               self.df['cum_capital']) * 100
        self.df['cum_return'] = self.df['normalized_port_position'] / 100

    def show_allocations(self):
        al_df = self.df.where(self.df.allocation > 0).dropna()
        return (al_df)
        
    def stats(self):
        df = deepcopy(self.df)
        stats = {}
        stats = stats | deepcopy(self.__dict__)
        del stats['df'] 
        stats['capital allocated'] = df.allocation.sum()
        stats['BTC allocated'] = df.BTC_tx.sum()
        stats['max portfolio value'] = df.portfolio_position.max()
        stats['final portfolio value'] = df.portfolio_position[-1]
        # Calculate Return on Invested Capital
        stats['ROIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated']) - 1
            )
        # Calculate Multiple of Invested Capital
        stats['MOIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated'])
            )
        
        stats['number_points'] = df['change'].count()
        stats['volatility_daily'] = df['change'].std()
        stats['annualization_factor'] = annualization_factor(df)
        stats['volatility_annual'] = df['change'].std() * annualization_factor(df)**.5
        stats['return_annual'] = ((df['cum_return'][-1]) ** (annualization_factor(df) / df['change'].count())) -1
    
        stats['sharpe_ratio'] = (stats['return_annual'] -
                             self.risk_free_rate) / stats['volatility_annual']
        Y = df['normalized_port_position'].pct_change().dropna()
        stats['hist_VaR_95'] = rp.RiskFunctions.VaR_Hist(Y, alpha=0.05)
        stats['max_DD_abs'] = rp.RiskFunctions.MDD_Rel(Y)
        stats['avg_DD_abs'] = rp.RiskFunctions.ADD_Rel(Y)
        stats['DaR_DD_abs_95'] = rp.RiskFunctions.DaR_Rel(Y, alpha=0.05)
        # Bitcoin stats
        stats['BTC_initial'] = df['close'][0]
        stats['BTC_final'] = df['close'][-1]
        stats['BTC_MOIC'] = df['close'][-1] / df['close'][0]
        stats['BTC_period'] = (stats['BTC_final'] / stats['BTC_initial']) - 1
        
        # Compared to BTC - how much more or less $100 invested in the 
        # allocation strategy resulted compared to $100 in BTC
        stats['outperformance_DV100'] = (stats['MOIC'] * 100) - (stats['BTC_MOIC'] * 100)
        
        #  Cost Basis Statistics
        stats['weighted_avg_cost'] =  (stats['capital allocated'] / 
                                       stats['BTC allocated'])
        txs = self.show_allocations()
        stats['avg_buy_price'] = txs['close'].mean()
        stats['max_buy_price'] = txs['close'].max()
        stats['min_buy_price'] = txs['close'].min()
        stats['first_buy_price'] = txs['close'][0]
        stats['lump_sum_beats_avg'] = (stats['first_buy_price'] < stats['avg_buy_price'])
        return (stats)
    
def run_through_time(alloc):
    # Do the same allocation for every start date 
    # in the dataframe. This basically shortens the dataframe by 1 day
    # on every loop and stores the results.
    # And stores the results in a new df with stats.
    
    # Stores initial values
    stats = {}
    stats['inputs'] = stats | deepcopy(alloc.__dict__)
    
    # Store allocations outputs
    stats['outputs'] = {}
    # Remove the df to save memory
    del stats['inputs']['df'] 
    tmp_alloc = deepcopy(alloc)  # Creates a copy of Alloc Instance

    results = []    
    # Will loop from start date until there aren't enough periods to allocate 
    while True:
        try:
            tmp_alloc.allocate_capital()
            results.append(tmp_alloc.stats())
            tmp_alloc.start_date = add_periods(tmp_alloc.start_date, 1, 'D')
        except Exception as e:
            break
    df = pd.DataFrame(results)
    
    # Create Outputs - sums, averages, depending on column
    stats['outputs']['number_of_allocations'] = df['frequency'].count()
    stats['outputs']['avg_BTC_allocated'] = df['BTC allocated'].mean()
    stats['outputs']['avg_lump_sum_beats_avg'] = df['lump_sum_beats_avg'].mean()
    stats['outputs']['avg_ROIC'] = df['ROIC'].mean()
    stats['outputs']['avg_sharpe'] = df['sharpe_ratio'].mean()
    stats['outputs']['avg_DD'] = df['avg_DD_abs'].mean()
    stats['outputs']['avg_DaR_DD_abs_95'] = df['DaR_DD_abs_95'].mean()
    stats['outputs']['avg_outperformance_DV100'] = df['outperformance_DV100'].mean()
    
    return (df, stats)
        
        

In [61]:
# Create a sample Instance of the AllocationManager and test results
btc_alloc = AllocationManager()
btc_alloc.capital = 100000
btc_alloc.allocation_periods = 2
btc_alloc.frequency = 'W'
btc_alloc.start_date = pd.to_datetime('2018-01-01', format='%Y-%m-%d')
btc_alloc.upfront_percent = 1
btc_alloc.allocate_capital()

In [62]:
btc_alloc.df

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2018-01-01,"13,444.88","13,850.49","13,921.53","12,877.67",-0.03,100000,7.44,7.44,"100,000.00",100000,100.00,1.00
2018-01-02,"14,754.13","13,444.88","15,306.13","12,934.16",0.10,0,0.00,7.44,"109,737.91",100000,109.74,1.10
2018-01-03,"15,156.62","14,754.09","15,435.01","14,579.71",0.03,0,0.00,7.44,"112,731.54",100000,112.73,1.13
2018-01-04,"15,180.08","15,156.49","15,408.66","14,244.67",0.00,0,0.00,7.44,"112,906.03",100000,112.91,1.13
2018-01-05,"16,954.78","15,180.08","17,126.95","14,832.36",0.12,0,0.00,7.44,"126,105.85",100000,126.11,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-23,"22,917.47","22,714.99","23,154.59","22,585.74",0.01,0,0.00,7.44,"170,455.00",100000,170.45,1.70
2023-01-24,"22,637.80","22,917.47","23,159.62","22,485.00",-0.01,0,0.00,7.44,"168,374.88",100000,168.37,1.68
2023-01-25,"23,066.28","22,637.80","23,801.04","22,354.60",0.02,0,0.00,7.44,"171,561.81",100000,171.56,1.72


In [63]:
# Show statistics
btc_alloc.stats()

{'frequency': 'W',
 'allocation_periods': 2,
 'upfront_percent': 1,
 'capital': 100000,
 'start_date': Timestamp('2018-01-01 00:00:00'),
 'end_date': Timestamp('2023-01-27 00:00:00'),
 'risk_free_rate': 0.05,
 'capital allocated': 100000,
 'BTC allocated': 7.437775569584853,
 'max portfolio value': 502415.343238467,
 'final portfolio value': 170442.2798864698,
 'ROIC': 0.704422798864698,
 'MOIC': 1.704422798864698,
 'number_points': 1853,
 'volatility_daily': 0.03908126062332537,
 'annualization_factor': 365,
 'volatility_annual': 0.746646435835947,
 'return_annual': 0.11074818180900592,
 'sharpe_ratio': 0.08136137654094888,
 'hist_VaR_95': 0.060564191134139356,
 'max_DD_abs': 0.8117602184914076,
 'avg_DD_abs': 0.46492173036533363,
 'DaR_DD_abs_95': 0.7704314506501746,
 'BTC_initial': 13444.88,
 'BTC_final': 22915.76,
 'BTC_MOIC': 1.7044227988646978,
 'BTC_period': 0.7044227988646978,
 'outperformance_DV100': 2.842170943040401e-14,
 'weighted_avg_cost': 13444.88,
 'avg_buy_price': 1344

In [64]:
btc_alloc.df

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2018-01-01,"13,444.88","13,850.49","13,921.53","12,877.67",-0.03,100000,7.44,7.44,"100,000.00",100000,100.00,1.00
2018-01-02,"14,754.13","13,444.88","15,306.13","12,934.16",0.10,0,0.00,7.44,"109,737.91",100000,109.74,1.10
2018-01-03,"15,156.62","14,754.09","15,435.01","14,579.71",0.03,0,0.00,7.44,"112,731.54",100000,112.73,1.13
2018-01-04,"15,180.08","15,156.49","15,408.66","14,244.67",0.00,0,0.00,7.44,"112,906.03",100000,112.91,1.13
2018-01-05,"16,954.78","15,180.08","17,126.95","14,832.36",0.12,0,0.00,7.44,"126,105.85",100000,126.11,1.26
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-23,"22,917.47","22,714.99","23,154.59","22,585.74",0.01,0,0.00,7.44,"170,455.00",100000,170.45,1.70
2023-01-24,"22,637.80","22,917.47","23,159.62","22,485.00",-0.01,0,0.00,7.44,"168,374.88",100000,168.37,1.68
2023-01-25,"23,066.28","22,637.80","23,801.04","22,354.60",0.02,0,0.00,7.44,"171,561.81",100000,171.56,1.72


In [65]:
d, stats = run_through_time(btc_alloc)
stats

{'inputs': {'frequency': 'W',
  'allocation_periods': 2,
  'upfront_percent': 1,
  'capital': 100000,
  'start_date': Timestamp('2018-01-01 00:00:00'),
  'end_date': Timestamp('2023-01-27 00:00:00'),
  'risk_free_rate': 0.05},
 'outputs': {'number_of_allocations': 1839,
  'avg_BTC_allocated': 9.328659939421073,
  'avg_lump_sum_beats_avg': 0.0,
  'avg_ROIC': 1.137733322933878,
  'avg_sharpe': 3.75602313361075,
  'avg_DD': 0.3419834489418764,
  'avg_DaR_DD_abs_95': 0.6609336966158488,
  'avg_outperformance_DV100': -6.25926716656532e-16}}

In [66]:
d

,frequency,allocation_periods,upfront_percent,capital,start_date,end_date,risk_free_rate,capital allocated,BTC allocated,max portfolio value,...,BTC_final,BTC_MOIC,BTC_period,outperformance_DV100,weighted_avg_cost,avg_buy_price,max_buy_price,min_buy_price,first_buy_price,lump_sum_beats_avg
0,W,2,1,100000,2018-01-01,2023-01-27,0.05,100000,7.44,"502,415.34",...,"22,915.76",1.70,0.70,0.00,"13,444.88","13,444.88","13,444.88","13,444.88","13,444.88",False
1,W,2,1,100000,2018-01-02,2023-01-27,0.05,100000,6.78,"457,832.08",...,"22,915.76",1.55,0.55,0.00,"14,754.13","14,754.13","14,754.13","14,754.13","14,754.13",False
2,W,2,1,100000,2018-01-03,2023-01-27,0.05,100000,6.60,"445,674.17",...,"22,915.76",1.51,0.51,0.00,"15,156.62","15,156.62","15,156.62","15,156.62","15,156.62",False
3,W,2,1,100000,2018-01-04,2023-01-27,0.05,100000,6.59,"444,985.40",...,"22,915.76",1.51,0.51,-0.00,"15,180.08","15,180.08","15,180.08","15,180.08","15,180.08",False
4,W,2,1,100000,2018-01-05,2023-01-27,0.05,100000,5.90,"398,407.65",...,"22,915.76",1.35,0.35,0.00,"16,954.78","16,954.78","16,954.78","16,954.78","16,954.78",False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1834,W,2,1,100000,2023-01-09,2023-01-27,0.05,100000,5.82,"134,269.98",...,"22,915.76",1.33,0.33,-0.00,"17,179.03","17,179.03","17,179.03","17,179.03","17,179.03",False
1835,W,2,1,100000,2023-01-10,2023-01-27,0.05,100000,5.73,"132,242.28",...,"22,915.76",1.31,0.31,0.00,"17,442.44","17,442.44","17,442.44","17,442.44","17,442.44",False
1836,W,2,1,100000,2023-01-11,2023-01-27,0.05,100000,5.57,"128,588.92",...,"22,915.76",1.28,0.28,0.00,"17,938.00","17,938.00","17,938.00","17,938.00","17,938.00",False
1837,W,2,1,100000,2023-01-12,2023-01-27,0.05,100000,5.31,"122,374.03",...,"22,915.76",1.22,0.22,0.00,"18,849.00","18,849.00","18,849.00","18,849.00","18,849.00",False


In [67]:
# Create summary HTML table
heading = f'<h1>Summary Results for DCA</h1>'
display(HTML(heading))
        
table_stats = f"""
        <h3>Inputs</h3>
        <table>
        <thead>
        """

for key, value in stats['inputs'].items():
    table_stats += f"""
            <tr>
                <td style='text-align: left;'>
                {key}
                </td>
                <td style='text-align: right;'>
                {value}
                </td>
            </tr>       
            """

table_stats += f"""
        </thead>
        </table>
        """
display(HTML(table_stats))


table_stats = f"""
        <h3>Outputs</h3>
        <table>
        <thead>
        """

for key, value in stats['outputs'].items():
    table_stats += f"""
            <tr>
                <td style='text-align: left;'>
                {key}
                </td>
                <td style='text-align: right;'>
                {"{0:,.2f}".format(value)}
                </td>
            </tr>       
            """

table_stats += f"""
        </thead>
        </table>
        """


        
display(HTML(table_stats))
        

# Chart Outperformance along time
# dates need to be in Epoch time for Highcharts
df_chart = deepcopy(d)
df_chart['start_date'] = (df_chart['start_date'] - datetime(1970, 1, 1)).astype(np.int64)
df_chart['start_date'] = (df_chart['start_date'] / 1000000).astype(np.int64)
x = df_chart['start_date'].to_list()
y = df_chart['outperformance_DV100'].to_list()
plot_chart(x, y, 'dates', 'outperformance', 'bar', 'outperf', 'Outperformance of $100 invested lumpsum compared to cost average')


frequency,W
allocation_periods,2
upfront_percent,1
capital,100000
start_date,2018-01-01 00:00:00
end_date,2023-01-27 00:00:00
risk_free_rate,0.05


number_of_allocations,"1,839.00"
avg_BTC_allocated,9.33
avg_lump_sum_beats_avg,0.00
avg_ROIC,1.14
avg_sharpe,3.76
avg_DD,0.34
avg_DaR_DD_abs_95,0.66
avg_outperformance_DV100,-0.00


In [68]:
# Show only the allocation periods
al_df = btc_alloc.show_allocations()
display(al_df)
# Check it adds to allocation amount
print("Total allocation:")
print(round(al_df.allocation.sum(), 8))
if (round(al_df.allocation.sum(), 8) == round(btc_alloc.capital, 8)):
    print ("Checks [OK]")
else:
    print ("[ERROR] - something went wrong")

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2018-01-01,"13,444.88","13,850.49","13,921.53","12,877.67",-0.03,"100,000.00",7.44,7.44,"100,000.00","100,000.00",100.00,1.00


Total allocation:
100000.0
Checks [OK]


### Run Simulations
Create different allocation scenarios

In [78]:
# CONSTANTS
# Define some static variables that will remain constant

# --------------
# Start Date
# --------------
# Data from 2010 has less meaning than recent data. For this analysis we can use
# more recent data. Using only data since 2017.
sim_start_date = pd.to_datetime('2021-11-01', format='%Y-%m-%d')

# Assuming $100,000 to be allocated
sim_allocated_capital = 100000

# Risk Free Rate (used to calculate sharpe ratio)
sim_risk_free_rate = 0.05


In [79]:
# Variables
# These will be the ranges / variables to be simulated

sim_frequencies = ['D', 'W', 'M']
sim_allocation_periods = {
    'D': [2, 10, 30, 60],
    'W': [2, 4, 8],
    'M': [2, 6, 12]
}
sim_upfront_percents = [0, 0.25, 0.5, 0.75, 1]

In [80]:
# Loop to gather results
print("Running scenarios...")
sim_results =  []
for freq in sim_frequencies:
    for all in sim_allocation_periods[freq]:
        for upfr in sim_upfront_percents:
            # Create allocation instance
            sim_alloc = AllocationManager()
            sim_alloc.capital = sim_allocated_capital
            sim_alloc.allocation_periods = all
            sim_alloc.frequency = freq
            sim_alloc.start_date = sim_start_date
            sim_alloc.upfront_percent = upfr
            sim_alloc.allocate_capital()
            print('Running scenario: ' + str(freq) + ' ' + str(upfr) + ' ' + str(all))
            # Run this allocation through time
            _, stats = run_through_time(sim_alloc)
            sim_results.append(stats['inputs'] | stats['outputs'])

sim_df = pd.DataFrame(sim_results)

Running scenarios...
Running scenario: D 0 2
Running scenario: D 0.25 2
Running scenario: D 0.5 2
Running scenario: D 0.75 2
Running scenario: D 1 2
Running scenario: D 0 10
Running scenario: D 0.25 10
Running scenario: D 0.5 10
Running scenario: D 0.75 10
Running scenario: D 1 10
Running scenario: D 0 30
Running scenario: D 0.25 30
Running scenario: D 0.5 30
Running scenario: D 0.75 30
Running scenario: D 1 30
Running scenario: D 0 60
Running scenario: D 0.25 60
Running scenario: D 0.5 60
Running scenario: D 0.75 60
Running scenario: D 1 60
Running scenario: W 0 2
Running scenario: W 0.25 2
Running scenario: W 0.5 2
Running scenario: W 0.75 2
Running scenario: W 1 2
Running scenario: W 0 4
Running scenario: W 0.25 4
Running scenario: W 0.5 4
Running scenario: W 0.75 4
Running scenario: W 1 4
Running scenario: W 0 8
Running scenario: W 0.25 8
Running scenario: W 0.5 8
Running scenario: W 0.75 8
Running scenario: W 1 8
Running scenario: M 0 2
Running scenario: M 0.25 2
Running scenario:

In [72]:
sim_results

[{'frequency': 'D',
  'allocation_periods': 2,
  'upfront_percent': 0,
  'capital': 100000,
  'start_date': Timestamp('2017-01-01 00:00:00'),
  'end_date': Timestamp('2023-01-27 00:00:00'),
  'risk_free_rate': 0.05,
  'number_of_allocations': 2216,
  'avg_BTC_allocated': 15.935510444595012,
  'avg_lump_sum_beats_avg': 0.5320397111913358,
  'avg_ROIC': 2.651743328258326,
  'avg_sharpe': 2.9867385260580583,
  'avg_DD': 0.3588276241293725,
  'avg_DaR_DD_abs_95': 0.6788851992322953,
  'avg_outperformance_DV100': -0.49694930888141026},
 {'frequency': 'D',
  'allocation_periods': 2,
  'upfront_percent': 0.25,
  'capital': 100000,
  'start_date': Timestamp('2017-01-01 00:00:00'),
  'end_date': Timestamp('2023-01-27 00:00:00'),
  'risk_free_rate': 0.05,
  'number_of_allocations': 2216,
  'avg_BTC_allocated': 15.924667485232513,
  'avg_lump_sum_beats_avg': 0.5320397111913358,
  'avg_ROIC': 2.649258581713918,
  'avg_sharpe': 2.8593804155232614,
  'avg_DD': 0.3586902068072247,
  'avg_DaR_DD_abs_9

In [73]:
sim_df

,frequency,allocation_periods,upfront_percent,capital,start_date,end_date,risk_free_rate,number_of_allocations,avg_BTC_allocated,avg_lump_sum_beats_avg,avg_ROIC,avg_sharpe,avg_DD,avg_DaR_DD_abs_95,avg_outperformance_DV100
0,D,2,0.00,100000,2017-01-01,2023-01-27,0.05,2216,15.94,0.53,2.65,2.99,0.36,0.68,-0.50
1,D,2,0.25,100000,2017-01-01,2023-01-27,0.05,2216,15.92,0.53,2.65,2.86,0.36,0.68,-0.75
2,D,2,0.50,100000,2017-01-01,2023-01-27,0.05,2216,15.94,0.53,2.65,2.99,0.36,0.68,-0.50
3,D,2,0.75,100000,2017-01-01,2023-01-27,0.05,2216,15.95,0.53,2.65,3.15,0.36,0.68,-0.25
4,D,2,1.00,100000,2017-01-01,2023-01-27,0.05,2216,15.96,0.00,2.66,3.35,0.36,0.68,-0.00
5,D,10,0.00,100000,2017-01-01,2023-01-27,0.05,2208,15.80,0.56,2.62,1.52,0.36,0.68,-4.48
6,D,10,0.25,100000,2017-01-01,2023-01-27,0.05,2208,15.84,0.56,2.63,1.63,0.36,0.68,-3.74
7,D,10,0.50,100000,2017-01-01,2023-01-27,0.05,2208,15.89,0.56,2.64,1.92,0.36,0.68,-2.49
8,D,10,0.75,100000,2017-01-01,2023-01-27,0.05,2208,15.94,0.56,2.65,2.41,0.36,0.68,-1.25
9,D,10,1.00,100000,2017-01-01,2023-01-27,0.05,2208,16.00,0.00,2.67,3.27,0.36,0.68,-0.00


In [74]:
sim_df.to_excel("output.xlsx") 

## Optimization

In [75]:
# Minimize DV100
def optimize_allocation(allocation_periods):
    # print(allocation_periods)
    if allocation_periods < 2:
        return 100000000000
    opt_alloc = AllocationManager()
    opt_alloc.capital = 100000
    opt_alloc.allocation_periods = int(allocation_periods)
    opt_alloc.frequency = 'W'
    opt_alloc.start_date = pd.to_datetime('2017-01-01', format='%Y-%m-%d')
    opt_alloc.upfront_percent = 0
    opt_alloc.allocate_capital()
    d, stats = run_through_time(opt_alloc)
    # Maximize DV100
    return (stats['outputs']['avg_outperformance_DV100'] * -1)

from scipy.optimize import minimize_scalar

# res = minimize_scalar(optimize_allocation, method='brent', options={'disp':1})
# res.fun
# res.x

In [76]:
# Chart showing allocation periods results
allocs = range(2, 24, 2)
results_x = []
results_y = []
for alloc in allocs:
    results_x.append(alloc)
    results_y.append(optimize_allocation(alloc) * -1)

In [77]:
plot_chart(results_x, results_y, f'DCA Allocation # of Weeks (0% upfront)', 'outperformance', 'bar', 'outperf', 'Outperformance of $100 invested lumpsum compared to cost average (since Jan, 2017)', dt=False)